## Update Legislation and Politicians Based on JSON File- Not For Notebook Jobs

##### Read all the Politicians stored in DynamoDB

In [2]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
import json
import pandas as pd

In [3]:
def query_Politicians_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('PoliticiansDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items
Current_DB_Politicians = query_Politicians_DB()

This is the Class for storing information

In [4]:
class CongressionalBillInfo:
    def __init__ (self, ID, Title, Summary, IntroducedDate, UpdatedDate, PDF_URL, Party, PolicyArea, Category, Session, Chamber,OriginChamber, Days, Aye, Nay, Not_Voting, Present, DemocratAyeCount, RepublicanAyeCount, RepublicanNayCount, DemocratNayCount, DemocratNotVotingCount, RepublicanNotVotingCount, Result):
        self.ID = ID
        self.Title = Title
        self.Summary = Summary
        self.IntroducedDate = IntroducedDate
        self.UpdatedDate = UpdatedDate
        self.PDF_URL = PDF_URL
        self.Party = Party
        self.PolicyArea = PolicyArea
        self.Category = Category
        self.Chamber = Chamber
        self.OriginChamber = OriginChamber
        self.Session = Session
        self.Days = Days
        self.Aye = Aye
        self.Nay = Nay
        self.Not_Voting = Not_Voting
        self.Present = Present
        self.DemocratAyeCount = DemocratAyeCount
        self.RepublicanAyeCount = RepublicanAyeCount
        self.DemocratNayCount = DemocratNayCount
        self.RepublicanNayCount = RepublicanNayCount
        self.DemocratNotVotingCount = DemocratNotVotingCount
        self.RepublicanNotVotingCount = RepublicanNotVotingCount
        self.Result = Result

Create lists of republican, democrat, and independents by alternateids. This will be used when going through legislation to categorize each vote properly.

In [5]:
republican_alt_id_list = []
democrat_alt_id_list = []
independent_alt_id_list = []

for politician in Current_DB_Politicians:
    
    if politician['Party'] == "Republican":
        republican_alt_id_list.append(politician['alternateID'])
    elif politician['Party'] == "Democratic":
        democrat_alt_id_list.append(politician['alternateID'])
    elif politician['Party'] == "Independent":
        independent_alt_id_list.append(politician['alternateID'])

In [6]:
import json

# Open the JSON file for reading
with open('Legislation-118-2023.json', 'r') as file:
    # Load the JSON data into a dictionary
    LegislationJSONList = json.load(file)

In [7]:
LegislationJSONList[0].keys()

dict_keys(['ID', 'Title', 'Summary', 'IntroducedDate', 'UpdatedDate', 'PDF_URL', 'Party', 'PolicyArea', 'Category', 'Chamber', 'OriginChamber', 'Session', 'Days', 'Aye', 'Nay', 'Not_Voting', 'Present', 'DemocratAyeCount', 'RepublicanAyeCount', 'DemocratNayCount', 'RepublicanNayCount', 'DemocratNotVotingCount', 'RepublicanNotVotingCount', 'Result'])

In [8]:
BillOBJList = []
for Bill in LegislationJSONList:
    CongressionalBillInfoOBJ = CongressionalBillInfo(ID = Bill['ID'], 
                                                     Category = Bill['Category'], 
                                                     Session = Bill['Session'], 
                                                     OriginChamber = Bill['OriginChamber'], 
                                                     Result= Bill['Result'], 
                                                     Title = Bill['Title'], 
                                                     Summary = Bill['Summary'], 
                                                     PolicyArea = Bill['PolicyArea'], 
                                                     IntroducedDate = Bill['IntroducedDate'], 
                                                     UpdatedDate = Bill['UpdatedDate'], 
                                                     PDF_URL = Bill['PDF_URL'], 
                                                     Party = Bill['Party'], 
                                                     Chamber = Bill['Chamber'], 
                                                     Days = Bill['Days'], 
                                                     Aye = Bill['Aye'], 
                                                     Nay = Bill['Nay'], 
                                                     Not_Voting = Bill['Not_Voting'], 
                                                     Present = Bill['Present'],
                                                     DemocratAyeCount = Bill['DemocratAyeCount'], 
                                                     RepublicanAyeCount = Bill['RepublicanAyeCount'], 
                                                     RepublicanNayCount = Bill['RepublicanNayCount'],
                                                     DemocratNayCount = Bill['DemocratNayCount'], 
                                                     DemocratNotVotingCount = Bill['DemocratNotVotingCount'], 
                                                     RepublicanNotVotingCount = Bill['RepublicanNotVotingCount'])
    BillOBJList.append(CongressionalBillInfoOBJ)

Create a class that also contains independent voting counts along with the 'Name_Party_State" dictionary lists where the key is the politician alternateID and then the rest is a list of the corresponding politician's name, state, and party they are associated with.

In [9]:
class CongressionalBillInfoWithExtendedVoteInfo:
    def __init__ (self, ID, Title, Summary, IntroducedDate, UpdatedDate, PDF_URL, Party, PolicyArea, Category, Session, Chamber,OriginChamber, Days, Aye, Nay, Not_Voting, Aye_Name_Party_State, Nay_Name_Party_State, Not_Voting_Name_Party_State, Present, DemocratAyeCount, RepublicanAyeCount, IndependentAyeCount, RepublicanNayCount, DemocratNayCount, IndependentNayCount, DemocratNotVotingCount, RepublicanNotVotingCount, IndependentNotVotingCount, Result):
        self.ID = ID
        self.Title = Title
        self.Summary = Summary
        self.IntroducedDate = IntroducedDate
        self.UpdatedDate = UpdatedDate
        self.PDF_URL = PDF_URL
        self.Party = Party
        self.PolicyArea = PolicyArea
        self.Category = Category
        self.Chamber = Chamber
        self.OriginChamber = OriginChamber
        self.Session = Session
        self.Days = Days
        self.Aye = Aye
        self.Nay = Nay
        self.Not_Voting = Not_Voting
        self.Aye_Name_Party_State = Aye_Name_Party_State
        self.Nay_Name_Party_State = Nay_Name_Party_State
        self.Not_Voting_Name_Party_State = Not_Voting_Name_Party_State
        self.Present = Present
        self.DemocratAyeCount = DemocratAyeCount
        self.RepublicanAyeCount = RepublicanAyeCount
        self.IndependentAyeCount = IndependentAyeCount
        self.DemocratNayCount = DemocratNayCount
        self.RepublicanNayCount = RepublicanNayCount
        self.IndependentNayCount = IndependentAyeCount
        self.DemocratNotVotingCount = DemocratNotVotingCount
        self.RepublicanNotVotingCount = RepublicanNotVotingCount
        self.IndependentNotVotingCount = IndependentNotVotingCount
        self.Result = Result

In [10]:
index_counter = 0
Full_Bill_OBJ_List = []

for obj in BillOBJList:
    
    Aye_Name_Party_State_Dictionary = {}
    Nay_Name_Party_State_Dictionary = {}
    Not_Voting_Name_Party_State_Dictionary = {}
    
    for pol_id in obj.Aye:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['id']) or (pol_id == full_pol_info_from_DB['alternateID']):
                Aye_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
        
    for pol_id in obj.Nay:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['id']) or (pol_id == full_pol_info_from_DB['alternateID']):
                Nay_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
    
    for pol_id in obj.Not_Voting:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['id']) or (pol_id == full_pol_info_from_DB['alternateID']):
                Not_Voting_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
                

    DemocratAyeCount = 0
    RepublicanAyeCount = 0
    IndependentAyeCount = 0

    DemocratNayCount = 0
    RepublicanNayCount = 0
    IndependentNayCount = 0

    DemocratNotVotingCount = 0
    RepublicanNotVotingCount = 0
    IndependentNotVotingCount = 0

    for politician_id in obj.Aye:

        if politician_id in republican_alt_id_list:
            RepublicanAyeCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratAyeCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentAyeCount+=1
    
    BillOBJList[index_counter].RepublicanAyeCount = RepublicanAyeCount
    BillOBJList[index_counter].DemocratAyeCount = DemocratAyeCount
    BillOBJList[index_counter].IndependentAyeCount = IndependentAyeCount

    for politician_id in obj.Nay:

        if politician_id in republican_alt_id_list:
            RepublicanNayCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratNayCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentNayCount+=1
    
    BillOBJList[index_counter].RepublicanNayCount = RepublicanNayCount
    BillOBJList[index_counter].DemocratNayCount = DemocratNayCount
    BillOBJList[index_counter].IndependentNayCount = IndependentNayCount

    for politician_id in obj.Not_Voting:

        if politician_id in republican_alt_id_list:
            RepublicanNotVotingCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratNotVotingCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentNotVotingCount+=1        

    BillOBJList[index_counter].RepublicanNotVotingCount = RepublicanNotVotingCount
    BillOBJList[index_counter].DemocratNotVotingCount = DemocratNotVotingCount
    BillOBJList[index_counter].IndependentNotVotingCount = IndependentNotVotingCount
    
    # print(LegislationJSONList[index_counter]['RepublicanAyeCount'])
    
    index_counter+=1
    
    CongressionalBillInfoWithExtendedVoteInfoOBJ = CongressionalBillInfoWithExtendedVoteInfo(ID = obj.ID, 
                                                                                             Category = obj.Category, 
                                                                                             Session = obj.Session, 
                                                                                             OriginChamber = obj.OriginChamber, 
                                                                                             Result= obj.Result, 
                                                                                             Title = obj.Title, 
                                                                                             Summary = obj.Summary, 
                                                                                             PolicyArea = obj.PolicyArea, 
                                                                                             IntroducedDate = obj.IntroducedDate, 
                                                                                             UpdatedDate = obj.UpdatedDate, 
                                                                                             PDF_URL = obj.PDF_URL, 
                                                                                             Party = obj.Party, 
                                                                                             Chamber = obj.Chamber, 
                                                                                             Days = obj.Days, 
                                                                                             Aye = obj.Aye,
                                                                                             Nay = obj.Nay,
                                                                                             Not_Voting = obj.Not_Voting, 
                                                                                             Aye_Name_Party_State = Aye_Name_Party_State_Dictionary, 
                                                                                             Nay_Name_Party_State = Nay_Name_Party_State_Dictionary, 
                                                                                             Not_Voting_Name_Party_State = Not_Voting_Name_Party_State_Dictionary,
                                                                                             Present = obj.Present,
                                                                                             DemocratAyeCount = DemocratAyeCount, 
                                                                                             RepublicanAyeCount = RepublicanAyeCount, 
                                                                                             IndependentAyeCount = IndependentAyeCount,
                                                                                             RepublicanNayCount = RepublicanNayCount, 
                                                                                             DemocratNayCount = DemocratNayCount,
                                                                                             IndependentNayCount = IndependentNayCount,
                                                                                             DemocratNotVotingCount = DemocratNotVotingCount,
                                                                                             RepublicanNotVotingCount = RepublicanNotVotingCount,
                                                                                             IndependentNotVotingCount = IndependentNotVotingCount)
    Full_Bill_OBJ_List.append(CongressionalBillInfoWithExtendedVoteInfoOBJ)

In [11]:
def write_to_CongressionalBillsDB(CongressionalBillInfoOBJ, dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")

    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.put_item(
       Item={
            'id': CongressionalBillInfoOBJ.ID,
            'Title': CongressionalBillInfoOBJ.Title,
            'Summary': CongressionalBillInfoOBJ.Summary,
            'IntroducedDate': CongressionalBillInfoOBJ.IntroducedDate,
            'UpdatedDate': CongressionalBillInfoOBJ.UpdatedDate,
            'PDF_URL': CongressionalBillInfoOBJ.PDF_URL,
            'Party': CongressionalBillInfoOBJ.Party,
            'PolicyArea': CongressionalBillInfoOBJ.PolicyArea,
            'Chamber': CongressionalBillInfoOBJ.Chamber,
            'OriginChamber': CongressionalBillInfoOBJ.OriginChamber,
            'Days': CongressionalBillInfoOBJ.Days,
            'Aye' : CongressionalBillInfoOBJ.Aye,
            'Nay' : CongressionalBillInfoOBJ.Nay,
            'Not_Voting' : CongressionalBillInfoOBJ.Not_Voting,
            'Aye_Name_Party_State': CongressionalBillInfoOBJ.Aye_Name_Party_State,
            'Nay_Name_Party_State': CongressionalBillInfoOBJ.Nay_Name_Party_State,
            'Not_Voting_Name_Party_State': CongressionalBillInfoOBJ.Not_Voting_Name_Party_State,
            'Present' : CongressionalBillInfoOBJ.Present,
            'DemocratAyeCount' : CongressionalBillInfoOBJ.DemocratAyeCount,
            'RepublicanAyeCount' : CongressionalBillInfoOBJ.RepublicanAyeCount,
            'IndependentAyeCount' : CongressionalBillInfoOBJ.IndependentAyeCount,
            'RepublicanNayCount' : CongressionalBillInfoOBJ.RepublicanNayCount,
            'DemocratNayCount' : CongressionalBillInfoOBJ.DemocratNayCount,
            'IndependentNayCount' : CongressionalBillInfoOBJ.IndependentNayCount,
            'DemocratNotVotingCount' : CongressionalBillInfoOBJ.DemocratNotVotingCount,
            'RepublicanNotVotingCount' : CongressionalBillInfoOBJ.RepublicanNotVotingCount,
            'IndependentNotVotingCount': CongressionalBillInfoOBJ.IndependentNotVotingCount,
            'Result': CongressionalBillInfoOBJ.Result,
            'Session': CongressionalBillInfoOBJ.Session,
            'Category': CongressionalBillInfoOBJ.Category
        }
    )
    return response

In [12]:
for Bill in Full_Bill_OBJ_List:
    write_to_CongressionalBillsDB(Bill)

### Write all Politician Information including Votes to S3 based on CongressionalBillsDB

In [27]:
import requests
import re
import ast
import operator
import regex
import os, shutil
import urllib.request
from PIL import Image
import pickle
import time
import sys
import gzip
import os

In [28]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
import json
import pandas as pd

#### Read all the legislation from the Congressional Bill DB

In [29]:
def query_Congress_Leg_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items

Prior_Voted_Legistlation = query_Congress_Leg_DB()

#### Read all the Politicians stored in DynamoDB

In [31]:
def query_Politicians_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('PoliticiansDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items
Current_DB_Politicians = query_Politicians_DB()

#### Write all the politician information with the corresponding legislation information on each voted bill to a JSON file and then save to S3

In [38]:
for politician in Current_DB_Politicians:
    
    pol_alt_id = politician['alternateID']
    pol_aye_vote_dict = {}
    pol_nay_vote_dict = {}
    pol_no_vote_dict = {}

    for legislation in Prior_Voted_Legistlation:
        
        for aye_legislation_pol in legislation['Aye_Name_Party_State'].keys():
            if aye_legislation_pol == pol_alt_id:
                pol_aye_vote_dict[legislation['id']] = legislation
                break
        
        for nay_legislation_pol in legislation['Nay_Name_Party_State']:
            if nay_legislation_pol == pol_alt_id:
                pol_nay_vote_dict[legislation['id']] = legislation
                break
        
        for not_voting_legislation_pol in legislation['Not_Voting_Name_Party_State']:
            if not_voting_legislation_pol == pol_alt_id:
                pol_no_vote_dict[legislation['id']] = legislation
                break
    politician['AyeVotedBills'] = pol_aye_vote_dict
    politician['NayVotedBills'] = pol_nay_vote_dict
    politician['NotVotedBills'] = pol_no_vote_dict
    
    # Specify the filename
    filename = "PoliticianJSONFolder/" + politician['id'] + '.json'
    # Convert Decimal to string during serialization

    # Open the file in write mode ('w') and write the dictionary
    with open(filename, 'w') as file:
        json.dump(politician, file, default = str, indent=4)  # Use 'indent' for pretty printing

    bucket_name = "moderateapps3bucket01959-dev"
    file_name_in_s3 = "public/" + filename

    # json_str = json.dumps(politician, default=str)
    # print(type(json_str))
    # json_data = json.loads(json_str)

    s3 = boto3.client('s3')
    with open(filename, "rb") as f:
        bucket = "moderateapps3bucket01959-dev"
        save_string = "public/" + filename
        s3.upload_fileobj(f, bucket, save_string)

#### Delete All the JSON files in the Folder

In [ ]:
# Specify the directory you want to clear
folder_path = 'PoliticianJSONFolder/'

# Loop through all files in the directory
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        # Check if it is a file and not a directory
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path) # This will delete the file
        # Uncomment below if you also want to remove subdirectories
        # elif os.path.isdir(file_path):
        #     shutil.rmtree(file_path) # Be careful with this, it deletes directories
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')
